In [1]:
# Setup logging
from timeseria import logger
logger.setup(level='INFO')

⚠️ This Notebook renders with limited features (e.g. no plots) if opened on GitHub, or as an untrusted Notebook in your local environment. Please either open it using [NBViewer](https://nbviewer.org/github/sarusso/Timeseria-notebooks/tree/master/notebooks/) or mark it as [trusted](https://stackoverflow.com/questions/44943646/jupyter-notebook-not-trusted).

# Advanced evaluation of  forecasting models


This notebook showcase how to perform advanced evaluations of forecasting models in [Timeseria](https://github.com/sarusso/Timeseria).

The forecaster we take as reference is a very simple (yet somehow powerful) periodic averages forecaster, which simply computes the periodicity in the data and then the average values within that period. It fits nearly instantaneously and it has the very same interfaces as its more powerful counterparts as the neural network-based ones, making it perfect for demonstrative purposes.

Let's start by loading an example time series:

In [2]:
from timeseria import storages
DATASET_PATH = '/'.join(storages.__file__.split('/')[0:-1]) + '/tests/test_data/csv/'
csv_storage = storages.CSVFileStorage(DATASET_PATH + 'temperature.csv')

In [3]:
timeseries = csv_storage.get()

Have a look at the time series we just loaded 

In [4]:
timeseries

Time series of #18331 points at variable resolution (~10m), from point @ 1546477200.0 (2019-01-03 01:00:00+00:00) to point @ 1557960000.0 (2019-05-15 22:40:00+00:00)

In [5]:
timeseries.plot()

[INFO] timeseria.plots: Aggregating by "10" for improved plotting


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Resample to one hour, and make data uniform

In [6]:
hourly_timeseries = timeseries.resample('1h')

[INFO] timeseria.transformations: Using auto-detected sampling interval: 600.0s
[INFO] timeseria.transformations: Resampled 18331 DataTimePoints in 3189 DataTimePoints


In [7]:
hourly_timeseries.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Instantiate and fit the model

In [8]:
from timeseria.models import PeriodicAverageForecaster

forecaster = PeriodicAverageForecaster()
forecaster.fit(hourly_timeseries)

[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"


Call the `predict()` function of the model. This returns key-value data with the prediction values

In [9]:
hourly_timeseries_with_forecast = forecaster.apply(hourly_timeseries, steps=24*3)

In [10]:
hourly_timeseries_with_forecast.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluate the forecaster. By default, window-based forecasters get evaluated on one step ahead and on a number of steps ahead equal to the window length, with RMSE and MAE metrics:

In [11]:
forecaster.evaluate(hourly_timeseries)

[INFO] timeseria.models.forecasters: Will evaluate model for [1, 24] steps ahead with metrics ['RMSE', 'MAE']


{'RMSE': 0.6187106383406303, 'MAE': 0.46844984996838346}

We can set custom steps ahedad and add or remove evalution metrics to the evaluation:

In [12]:
forecaster.evaluate(hourly_timeseries, steps=[1,2,3], metrics=['MAE', 'MAPE'])

[INFO] timeseria.models.forecasters: Will evaluate model for [1, 2, 3] steps ahead with metrics ['MAE', 'MAPE']


{'MAE': 0.40894241863588027, 'MAPE': 0.018088814494578456}

We can ask for the details when evaluating:

In [13]:
forecaster.evaluate(hourly_timeseries, steps=[1,2,3], metrics=['MAE', 'MAPE'], details=True)

[INFO] timeseria.models.forecasters: Will evaluate model for [1, 2, 3] steps ahead with metrics ['MAE', 'MAPE']


{'MAE_1_steps': 0.401049913670268,
 'MAPE_1_steps': 0.017737905360553242,
 'MAE_2_steps': 0.40913082876187573,
 'MAPE_2_steps': 0.018097067301057805,
 'MAE_3_steps': 0.416646513475497,
 'MAPE_3_steps': 0.018431470822124322,
 'MAE': 0.40894241863588027,
 'MAPE': 0.018088814494578456}

If evaluating for one step ahead only, we can also get back the evaluation time series

In [14]:
results = forecaster.evaluate(hourly_timeseries, steps=[1], metrics=['MAE', 'MAPE'], evaluation_timeseries=True)

[INFO] timeseria.models.forecasters: Will evaluate model for [1] steps ahead with metrics ['MAE', 'MAPE']


In [15]:
results

{'MAE': 0.401049913670268,
 'MAPE': 0.017737905360553242,
 'evaluation_timeseries': Time series of #3165 points at 1h resolution, from point @ 1546567200.0 (2019-01-04 02:00:00+00:00) to point @ 1557957600.0 (2019-05-15 22:00:00+00:00)}

...which we can then plot:

In [16]:
results['evaluation_timeseries'].plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We can also directly plot the evaluation (for one step ahead) and remove any error metrics to visually inspect the predictions:

In [17]:
forecaster.evaluate(hourly_timeseries, steps=[1], metrics=None, plot=True)

[INFO] timeseria.models.forecasters: Will evaluate model for [1] steps ahead with metrics []


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Lastly, Timeseria also supports cross-validating the forecasting models:

In [18]:
forecaster.cross_validate(hourly_timeseries, rounds=3)

[INFO] timeseria.models.base: Cross validation round #1 of 3: validate from 1546480800.0 (2019-01-03 02:00:00+00:00) to 1550307600.0 (2019-02-16 09:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"
[INFO] timeseria.models.forecasters: Will evaluate model for [1, 24] steps ahead with metrics ['RMSE', 'MAE']
[INFO] timeseria.models.base: Cross validation round #2 of 3: validate from 1550307600.0 (2019-02-16 09:00:00+00:00) to 1554134400.0 (2019-04-01 16:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"
[INFO] timeseria.models.forecasters: Will evaluate model for [1, 24] steps ahead with metrics ['RMSE', 'MAE']
[INFO] timeseria.models.base: Cross validation round #3 of 3: validate from 1554134400.0 (2019-04-01 16:00:00+00:00) to 1557957600.0 (2019-05-15 22:00:00+00:00), fit on t

{'RMSE_avg': 0.6473975548915724,
 'RMSE_stdev': 0.07280319905465943,
 'MAE_avg': 0.4963014373340906,
 'MAE_stdev': 0.05826778726031544}

In the cross validation, we can pass arguments to the fit and evlauate calls by just prepending with a `fit_` or `evaluate_`:

In [19]:
forecaster.cross_validate(hourly_timeseries, rounds=3, evaluate_metrics=['MAPE'], evaluate_steps=[1])

[INFO] timeseria.models.base: Cross validation round #1 of 3: validate from 1546480800.0 (2019-01-03 02:00:00+00:00) to 1550307600.0 (2019-02-16 09:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"
[INFO] timeseria.models.forecasters: Will evaluate model for [1] steps ahead with metrics ['MAPE']
[INFO] timeseria.models.base: Cross validation round #2 of 3: validate from 1550307600.0 (2019-02-16 09:00:00+00:00) to 1554134400.0 (2019-04-01 16:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"
[INFO] timeseria.models.forecasters: Will evaluate model for [1] steps ahead with metrics ['MAPE']
[INFO] timeseria.models.base: Cross validation round #3 of 3: validate from 1554134400.0 (2019-04-01 16:00:00+00:00) to 1557957600.0 (2019-05-15 22:00:00+00:00), fit on the rest.
[INFO] timese

{'MAPE_avg': 0.01913598689488296, 'MAPE_stdev': 0.0026036697802568477}